In [17]:
# ! pip install tqdm

In [2]:
# Libraries
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import animation

from IPython.display import HTML
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
# from hda import * # this is the PYTHON version

import numpy as np
import pandas as pd
from tqdm import tqdm

In [19]:
#Sea depth data
sea_d = 'dataSwell/olas'
sea_d = xr.open_dataset(sea_d)
sea_d


<xarray.Dataset>
Dimensions:    (latitude: 101, time: 55, longitude: 703)
Coordinates:
  * latitude   (latitude) float64 58.42 58.5 58.58 58.67 ... 66.58 66.67 66.75
  * time       (time) datetime64[ns] 2019-01-01T03:00:00 ... 2019-01-07T21:00:00
  * longitude  (longitude) float64 -58.42 -58.33 -58.25 ... -0.08333 0.0 0.08333
Data variables:
    VHM0_SW1   (time, latitude, longitude) float32 ...
    VTM01_SW1  (time, latitude, longitude) float32 ...
Attributes: (12/31)
    Conventions:                               CF-1.6
    time_coverage_start:                       20211115-03:00:00
    time_coverage_end:                         20211116-00:00:00
    date_created:                              20211106-07:39:00
    product_type:                              forecast
    product:                                   GLOBAL_ANALYSIS_FORECAST_WAV_0...
    ...                                        ...
    FROM_ORIGINAL_FILE__geospatial_lat_step:   0.08333588
    FROM_ORIGINAL_FILE__geospatial_lat_units:  degree
    _CoordSysBuilder:                          ucar.nc2.dataset.conv.CF1Conve...
    comment:                                   
    source:                                     
    history:                                   Data extracted from dataset ht...

In [30]:
wave_height = sea_d["VHM0_SW1"]
wave_height_df = wave_height.to_dataframe(name= "height", dim_order=None)

In [33]:
wave_height_df.to_csv("wave_height.csv")

In [34]:
wave_period = sea_d["VTM01_SW1"]
wave_period_df = wave_period.to_dataframe(name= "period", dim_order=None)
wave_period_df.to_csv("wave_period.csv")

In [23]:
# Power calculation 
def wavesPower(H = 5, Te = 10, dens = 1025, g = 9.8, maxH = 8):
    """
    P  = wave energy flux per unit of wave-crest length
    H = the significant wave height in m
    Te = the wave energy period in s
    dens = the water density
    g  = the acceleration by gravity
    
    When the significant wave height is given in meters, 
    and the wave period in seconds, the result is the wave power in kilowatts (kW) 
    per meter of wave front length.
    """
    if H < maxH:
        P = dens*g**2*H**2*Te/64*3.14
    else:
        P = 0
    return P

In [37]:
# Possible power calculation 
dens=1025
g=9.8
maxH =8
wave_heigh = pd.read_csv("wave_height.csv", delimiter=',', decimal='.')
wave_period = pd.read_csv("wave_period.csv", delimiter=',', decimal='.')

In [39]:
wave_period

,time,latitude,longitude,period
0,2019-01-01 03:00:00,58.416667,-58.416667,9.62
1,2019-01-01 03:00:00,58.416667,-58.333333,9.62
2,2019-01-01 03:00:00,58.416667,-58.250000,9.62
3,2019-01-01 03:00:00,58.416667,-58.166667,9.62
4,2019-01-01 03:00:00,58.416667,-58.083333,9.63
...,...,...,...,...
3905160,2019-01-07 21:00:00,66.750000,-0.250000,8.74
3905161,2019-01-07 21:00:00,66.750000,-0.166667,8.86
3905162,2019-01-07 21:00:00,66.750000,-0.083333,9.01
3905163,2019-01-07 21:00:00,66.750000,0.000000,9.15


In [45]:
wave_heigh

,time,latitude,longitude,height,energy
0,2019-01-01 03:00:00,58.416667,-58.416667,0.77,2.794812
1,2019-01-01 03:00:00,58.416667,-58.333333,0.78,2.867876
2,2019-01-01 03:00:00,58.416667,-58.250000,0.78,2.867876
3,2019-01-01 03:00:00,58.416667,-58.166667,0.79,2.941882
4,2019-01-01 03:00:00,58.416667,-58.083333,0.79,2.944940
...,...,...,...,...,...
3905160,2019-01-07 21:00:00,66.750000,-0.250000,2.28,22.262668
3905161,2019-01-07 21:00:00,66.750000,-0.166667,2.32,23.367151
3905162,2019-01-07 21:00:00,66.750000,-0.083333,2.37,24.798052
3905163,2019-01-07 21:00:00,66.750000,0.000000,2.43,26.474615


In [41]:
out_energy = []

for row in tqdm(range(len(wave_period))):
    H = wave_heigh["height"][row]
    Te = wave_period["period"][row]
    totalEnergy = 0.49 * H**2 * Te
    out_energy += [totalEnergy]

100%|██████████| 3905165/3905165 [01:02<00:00, 61997.95it/s]


In [42]:
wave_heigh["energy"] = out_energy


In [44]:
max_energy = max(wave_heigh["energy"])
max_energy

233.75717906359617

In [49]:
wave_heigh=wave_heigh[~wave_heigh["energy"].isnull()]
wave_heigh.to_csv("Energy_waves_m.csv")